In [ ]:
from IPython.display import clear_output
import random
import numpy as np

DEALER_THRESHOLD = 16
BLACKJACK = 21

def genDeck(shuffle=True):
    deck = np.array([[11 if x == 1 else 10 if x > 10 else x] * 4 for x in range(1, 14)] * 6)
    deck = deck.flatten()
    if shuffle:
        random.shuffle(deck)
    return list(deck)

def draw_card(deck):
    index = random.randint(0, len(deck) - 1)
    return deck.pop(index)

def sum_hand(hand):
    sum_value = sum(hand)
    num_aces = hand.count(11)

    while sum_value > BLACKJACK and num_aces:
        sum_value -= 10  # Convert an Ace from 11 to 1
        num_aces -= 1

    return sum_value

def dealer_logic(dealer_hand, deck):
    if sum_hand(dealer_hand) <= DEALER_THRESHOLD:
        card = draw_card(deck)
        print(f"Dealer drew: {card}")
        dealer_hand.append(card)
    else:
        print("Dealer stands")

def evaluate(player_stood, user_hand, dealer_hand):
    user_sum, dealer_sum = sum_hand(user_hand), sum_hand(dealer_hand)

    if dealer_sum > BLACKJACK:
        print(f"Dealer bust with: {dealer_sum, dealer_hand}")
        return True
    elif user_sum > BLACKJACK:
        print(f"Player bust with: {user_sum, user_hand}")
        return False
    elif dealer_sum == BLACKJACK:
        print(f"Dealer won with blackjack: {dealer_hand}")
        return False
    elif user_sum == BLACKJACK:
        print(f"Player won with blackjack: {user_hand}")
        return True
    elif dealer_sum > DEALER_THRESHOLD:
        if not player_stood:
            return None
        if dealer_sum > user_sum:
            print(f"Dealer won with: {dealer_sum, dealer_hand}")
            return False
        else:
            print(f"Player won with: {user_sum, user_hand}")
            return True
    else:
        return None

def play_round(deck):
    dealer_hand, user_hand = [draw_card(deck)], [draw_card(deck)]
    result, player_stood = None, False

    while result is None:
        print("Player's hand: ", user_hand)
        print("Dealer's hand: ", dealer_hand)

        while True:
            clear_output(wait=True)
            user_input = input("0 to stand, 1 to draw: ")
            if user_input.isdigit() and user_input in ['0', '1']:
                user_input = int(user_input)
                break
            else:
                print("Invalid input. Please enter 0 or 1.")

        if user_input == 1:
            card = draw_card(deck)
            print(f"Player drew: {card}")
            user_hand.append(card)
            player_stood = False

        if user_input == 0:
            print(f"Player stands with: {user_hand}")
            player_stood = True
            while result is None:
                dealer_logic(dealer_hand, deck)
                result = evaluate(player_stood, user_hand, dealer_hand)
            break
            

        dealer_logic(dealer_hand, deck)
        result = evaluate(player_stood, user_hand, dealer_hand)
    
    return result


In [ ]:
full_deck = genDeck()

def play(prev_state, action, deck):

    done = False
    reward = 0
    result = None
    
    dealer_hand, user_hand = prev_state
    if action == 1:
        card = draw_card(deck)
        user_hand.append(card)
        print(f"Player drew: {card}")
        dealer_logic(dealer_hand, deck)
        result = evaluate(False, user_hand, dealer_hand)
        if result is not None:
            done = True

    if action == 0:
        while result is None:
            dealer_logic(dealer_hand, deck)
            result = evaluate(True, user_hand, dealer_hand)
        done = True
    
    diff = np.abs(sum_hand(dealer_hand) - sum_hand(user_hand))

    if result == True:
        reward += 100 + diff
    elif result == False:
        reward -= 100 - diff
    
    state = (dealer_hand, user_hand)

    return state, reward, done

In [ ]:
play(([10, 6], [3, 8]), 1, full_deck)

In [ ]:
np.random.seed(42)

# State space size
player_sum_space_size = 32  # 0 to 21 inclusive
dealer_upcard_space_size = 32  # 0 to 21 inclusive

# Action space size
action_space_size = 2  # 0 for stand, 1 for draw

# Initialize Q-table with small random values
q_table = np.random.rand(player_sum_space_size, dealer_upcard_space_size, action_space_size) * 0.01
print(q_table[18][17])

def learn(ALPHA, GAMMA, q_value, max_next_q_value, reward):
    return (1 - ALPHA) * q_value + ALPHA * (reward + GAMMA * max_next_q_value)

In [4]:
episodes = 10000
epochs = 100

ALPHA = 0.1
GAMMA = 0.1
EPSILON = 0.20

results = []
for epoch in range(epochs):
    epoch_results = []
    deck = genDeck()

    state = ([draw_card(deck)], [draw_card(deck)])

    for i in range(episodes):
        old_state = state
        action = np.argmax(q_table[sum_hand(state[0])][sum_hand(state[1])])

        if np.random.uniform(0, 1) < EPSILON:
            action = random.randint(0, 1)

        state, reward, done = play(state, action, deck)

        q_table[sum_hand(state[0])][sum_hand(state[1])] = learn(ALPHA, GAMMA, np.max(q_table[sum_hand(old_state[0])][sum_hand(old_state[1])]), np.max(q_table[sum_hand(state[0])][sum_hand(state[1])]), reward)
        
        if done:
            win = True if reward >= 100 else False
            epoch_results.append(win)
            state = ([draw_card(deck)], [draw_card(deck)])
            

        if i % 10 == 0:
            deck = genDeck()
    results.append(epoch_results)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

#print(results)
wr = [sum(result) / len(result) for result in results]
#print(wr)
plt.plot(np.linspace(1, epochs, epochs), wr)
print(wr[0], wr[-1])

In [ ]:
deck = genDeck()
print(len(deck))
results = []
i = 0
for _ in range(5):
    i += 1
    print(f"round no.: {i}")
    results.append(play_round(deck))
print(results)
win_rate = len([result for result in results if result == True]) / len(results) * 100
print(f"{win_rate:.2f}% user winrate")

In [ ]:
def generate_computer_decision(probability):
    return random.choices([0, 1], weights=[1 - probability, probability], k=1)[0]

def play_round(deck, computer_probability):
    dealer_hand, user_hand = [draw_card(deck)], [draw_card(deck)]
    result, player_stood = None, False

    while result is None:
        print("Player's hand: ", user_hand)
        print("Dealer's hand: ", dealer_hand)

        # Generate computer decision
        computer_input = generate_computer_decision(computer_probability)
        print(f"Computer chose: {computer_input}")

        if computer_input == 1:
            card = draw_card(deck)
            print(f"Player drew: {card}")
            user_hand.append(card)
            player_stood = False

        if computer_input == 0:
            print(f"Player stands with: {user_hand}")
            player_stood = True

        dealer_logic(dealer_hand, deck)
        result = evaluate(player_stood, user_hand, dealer_hand)

    return result
full_sub_results = []
results = []
for i in range(1, 101):  # Range from 1 to 100 (inclusive)
    deck = genDeck()
    print(len(deck))
    current_probability = i / 100.0  # Convert to float and scale to the desired range
    print(f"current probability: {current_probability}")
    sub_results = []
    for _ in range(1000):
        sub_results.append(play_round(genDeck(), current_probability)) # p(x = 1)
    full_sub_results.append(sub_results)
    # print(results)
    win_rate = len([result for result in sub_results if result == True]) / len(sub_results) * 100
    print(f"{win_rate:.2f}% user winrate")
    results.append((current_probability, win_rate))

print(results)


In [ ]:
def flatten_list(nested_list):
    result = []
    for item in nested_list:
        if isinstance(item, list):
            result.extend(flatten_list(item))
        else:
            result.append(item)
    return result
flattened_list = flatten_list(full_sub_results)


In [ ]:
print(len(flattened_list))

In [ ]:
sorted_list = sorted(results, key=lambda x: x[1], reverse=True)

print(sorted_list)


In [ ]:
import matplotlib.pyplot as plt

second_values = [x[1] for x in results]

# Plotting the histogram
plt.hist(second_values, bins=10, color='blue', edgecolor='black')

# Adding labels and title
plt.xlabel('Second Values')
plt.ylabel('Frequency')
plt.title('Histogram of Second Values')

# Display the plot
plt.show()

print(f"mean wr: {sum(second_values) / len(second_values)}")